> CART分类树采用**Gini指数**来进行特征选择
> 完整的CART算法包括特征选择、决策树生成和决策树剪枝三个部分。
![微信图片_20200518122233.jpg](https://user-gold-cdn.xitu.io/2020/5/18/172260521433a2a1?w=1080&h=413&f=jpeg&s=53046)
> CART是在**给定输入随机变量X条件下输出随机变量Y的条件概率分布**的学习方法。CART算法通过选择最优特征和特征值进行划分，将输入空间也就是特征空间划分为有限个单元，并在这些单元上确定预测的概率分布，也就是在输入给定的条件下输出条件概率分布。
>
> CART算法主要包括**回归树和分类树**两种。回归树用于**目标变量为连续型的建模任务，其特征选择准则用的是平方误差最小准则**。分类树用于**目标变量为离散型的的建模任务，其特征选择准则用的是基尼指数(Gini Index)**，这也有别于此前ID3的信息增益准则和C4.5的信息增益比准则。无论是回归树还是分类树，其算法核心都在于**递归地选择最优特征构建决策树**。
>
> 除了选择最优特征构建决策树之外，CART算法还包括另外一个重要的部分：**剪枝**。剪枝可以视为决策树算法的一种**正则化手段**，作为一种基于规则的非参数监督学习方法，决策树在训练很容易过拟合，导致最后生成的决策树泛化性能不高。
> 另外，CART作为一种单模型，也是GBDT的基模型。当很多棵CART分类树或者回归树集成起来的时候，就形成了GBDT模型。
> 
> Gini指数是针对概率分布而言的。假设在一个分类问题中有K个类，样本属于第k个类的概率$P_k$，则该样本概率分布的基尼指数为
<center>$Gini(P)=\sum_{k=1}^{n}{(P_k*(1-P_k)}$ </center>
[参考文章](https://mp.weixin.qq.com/s/jdUQIPM2AhAh7rzl1DPgIQ)

# 获取数据


In [28]:
import pandas as pd 
# df=pd.read_excel('sales_data.xls').iloc[:,1:]
# df[["天气","销量"]]
df=pd.read_excel('bank-data.xls').iloc[:,1:]
car_no=df[df.car=="YES"]
pep_no=df[(df.car=="YES") & (df.pep=="NO")]
len(pep_no)/len(car_no)

0.5337837837837838

# 定义Gini指数的计算函数

In [7]:
def gini(ele):
    prob=[ele.count(i)/len(ele) for i in set(ele)]
    return sum(i*(1-i) for i in prob)
gini(df["pep"].tolist())

0.49624444444444443

# 定义根据特征分割数据框的函数

In [8]:
def split_df(df,label):
    result={}
    for k in df[label].unique():
        result[k]=df[df[label]==k]
    return result
split_df(df,'pep')

{'YES':      age     sex      region    income married children  car save_act  \
 0     48  FEMALE  INNER_CITY  17546.00      NO      YES   NO       NO   
 5     57  FEMALE        TOWN  37869.60     YES       NO   NO      YES   
 6     22    MALE       RURAL   8877.07      NO      YES   NO       NO   
 12    44  FEMALE        TOWN  15735.80     YES      YES   NO      YES   
 13    66  FEMALE        TOWN  55204.70     YES      YES  YES      YES   
 ..   ...     ...         ...       ...     ...      ...  ...      ...   
 587   43  FEMALE        TOWN  31273.80      NO      YES  YES      YES   
 591   40  FEMALE  INNER_CITY  31473.90      NO       NO   NO      YES   
 593   65    MALE    SUBURBAN  51417.00     YES       NO   NO      YES   
 597   31  FEMALE        TOWN  15976.30     YES      YES  YES      YES   
 599   38    MALE        TOWN  26671.60      NO      YES  YES       NO   
 
     current_act mortgage  pep  
 0            NO       NO  YES  
 5           YES       NO  YES  
 6  

# 根据Gini指数和条件Gini指数计算递归选择最优特征

In [9]:
def choose_best_col(df,label):
    cols=[c for c in df.columns if c!=label]
    min_gini,min_split_df,best_col=999,None,""
    for col in cols:
        gini_A=.0 
        splited_subset=split_df(df,col)
        for split_v,split_data in splited_subset.items():
            gini_K=gini(split_data[label].tolist())
            gini_A+=(len(split_data)/len(df))*gini_K
        if min_gini>gini_A:
            min_gini,min_split_df,best_col=gini_A,splited_subset,col 
    return min_gini,min_split_df,best_col
choose_best_col(df,"pep")

(0.0,
 {17546.0:    age     sex      region   income married children car save_act current_act  \
  0   48  FEMALE  INNER_CITY  17546.0      NO      YES  NO       NO          NO   
  
    mortgage  pep  
  0       NO  YES  ,
  30085.1:    age   sex region   income married children  car save_act current_act  \
  1   40  MALE   TOWN  30085.1     YES       NO  YES       NO         YES   
  
    mortgage pep  
  1      YES  NO  ,
  16575.4:    age     sex      region   income married children  car save_act  \
  2   51  FEMALE  INNER_CITY  16575.4     YES      YES  YES      YES   
  
    current_act mortgage pep  
  2         YES       NO  NO  ,
  20375.4:    age     sex region   income married children car save_act current_act  \
  3   23  FEMALE   TOWN  20375.4     YES       NO  NO       NO         YES   
  
    mortgage pep  
  3       NO  NO  ,
  50576.3:    age     sex region   income married children car save_act current_act  \
  4   57  FEMALE  RURAL  50576.3     YES      YES  NO    

# 定义CART分类树的构建过程

In [10]:
class CartTree:
    class Node:
        def __init__(self,name):
            self.name,self.connections=name,{}
        def connect(self,label,node):
            self.connections[label]=node
    def __init__(self,df,label):
        self.df,self.columns,self.label,self.root=df,df.columns,label,self.Node('Root')
    def construct(self,parent_node,parent_con_label,df,columns):
        min_gini,min_split_df,best_col=choose_best_col(df[columns],self.label)
        if not best_col:
            node=self.Node(df[self.label].iloc[0])
            parent_node.connect(parent_con_label,node)
        else:
            node=self.Node(best_col)
            parent_node.connect(parent_con_label,node)
            new_columns=[c for c in columns if c!=best_col]
            for split_v,split_data in min_split_df.items():
                self.construct(node,split_v,split_data,new_columns)
    def construct_tree(self):
        self.construct(self.root,"Root",self.df,self.columns)
    def print_tree(self,root,tabs):
        print(f"{tabs}({root.name})")
        for label ,node in root.connections.items():
            print(f"{tabs}\t| <{label}>")
            self.print_tree(node,tabs+"\t\t|")

tree=CartTree(df,"pep")
tree.construct_tree()
tree.print_tree(tree.root,"| ")


| (Root)
| 	| <Root>
| 		|(income)
| 		|	| <17546.0>
| 		|		|(age)
| 		|		|	| <48>
| 		|		|		|(sex)
| 		|		|		|	| <FEMALE>
| 		|		|		|		|(region)
| 		|		|		|		|	| <INNER_CITY>
| 		|		|		|		|		|(married)
| 		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|(children)
| 		|		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|		|(car)
| 		|		|		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|		|		|(save_act)
| 		|		|		|		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|		|		|		|(current_act)
| 		|		|		|		|		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|		|		|		|		|(mortgage)
| 		|		|		|		|		|		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|		|		|		|		|		|(YES)
| 		|	| <30085.1>
| 		|		|(age)
| 		|		|	| <40>
| 		|		|		|(sex)
| 		|		|		|	| <MALE>
| 		|		|		|		|(region)
| 		|		|		|		|	| <TOWN>
| 		|		|		|		|		|(married)
| 		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|(children)
| 		|		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|		|(car)
| 		|		|		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|		|		|(save_act)
| 		|		|		|		|		|		|		|		|	| <NO>


| 		|		|		|		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|		|		|		|(current_act)
| 		|		|		|		|		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|		|		|		|		|(mortgage)
| 		|		|		|		|		|		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|		|		|		|		|		|(NO)
| 		|	| <21796.6>
| 		|		|(age)
| 		|		|	| <41>
| 		|		|		|(sex)
| 		|		|		|	| <FEMALE>
| 		|		|		|		|(region)
| 		|		|		|		|	| <INNER_CITY>
| 		|		|		|		|		|(married)
| 		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|(children)
| 		|		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|		|(car)
| 		|		|		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|		|		|(save_act)
| 		|		|		|		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|		|		|		|(current_act)
| 		|		|		|		|		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|		|		|		|		|(mortgage)
| 		|		|		|		|		|		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|		|		|		|		|		|(NO)
| 		|	| <63130.1>
| 		|		|(age)
| 		|		|	| <67>
| 		|		|		|(sex)
| 		|		|		|	| <FEMALE>
| 		|		|		|		|(region)
| 		|		|		|		|	| <SUBURBAN>
| 		|		|		|		|		|(married)
| 		|	

| 		|		|		|		|		|		|		|		|		|(current_act)
| 		|		|		|		|		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|		|		|		|		|(mortgage)
| 		|		|		|		|		|		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|		|		|		|		|		|(YES)
| 		|	| <14642.2>
| 		|		|(age)
| 		|		|	| <29>
| 		|		|		|(sex)
| 		|		|		|	| <MALE>
| 		|		|		|		|(region)
| 		|		|		|		|	| <INNER_CITY>
| 		|		|		|		|		|(married)
| 		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|(children)
| 		|		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|		|(car)
| 		|		|		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|		|		|(save_act)
| 		|		|		|		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|		|		|		|(current_act)
| 		|		|		|		|		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|		|		|		|		|(mortgage)
| 		|		|		|		|		|		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|		|		|		|		|		|(YES)
| 		|	| <15933.3>
| 		|		|(age)
| 		|		|	| <48>
| 		|		|		|(sex)
| 		|		|		|	| <FEMALE>
| 		|		|		|		|(region)
| 		|		|		|		|	| <INNER_CITY>
| 		|		|		|		|		|(married)
| 		|		|		|		|		|	| <YES>
| 		|		|		|		|

| 		|		|		|		|		|		|		|		|		|(current_act)
| 		|		|		|		|		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|		|		|		|		|(mortgage)
| 		|		|		|		|		|		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|		|		|		|		|		|(NO)
| 		|	| <12623.4>
| 		|		|(age)
| 		|		|	| <29>
| 		|		|		|(sex)
| 		|		|		|	| <MALE>
| 		|		|		|		|(region)
| 		|		|		|		|	| <SUBURBAN>
| 		|		|		|		|		|(married)
| 		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|(children)
| 		|		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|		|(car)
| 		|		|		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|		|		|(save_act)
| 		|		|		|		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|		|		|		|(current_act)
| 		|		|		|		|		|		|		|		|		|	| <YES>
| 		|		|		|		|		|		|		|		|		|		|(mortgage)
| 		|		|		|		|		|		|		|		|		|		|	| <NO>
| 		|		|		|		|		|		|		|		|		|		|		|(NO)
| 		|	| <23818.6>
| 		|		|(age)
| 		|		|	| <25>
| 		|		|		|(sex)
| 		|		|		|	| <MALE>
| 		|		|		|		|(region)
| 		|		|		|		|	| <INNER_CITY>
| 		|		|		|		|		|(married)
| 		|		|		|		|		|	| <YES>
| 		|		|		|		|		|